# Генерация датасетов для обучения LLM

Этот notebook показывает как генерировать датасеты для SFT обучения.

**Ключевые возможности:**
- **81 тип задач** (40 математических + 41 физическая)
- **`reasoning_mode=True`** — генерация в формате `<think>...</think><answer>...</answer>`
- Двуязычность (ru/en), 10 уровней сложности
- Форматы: SFT, Chat, JSONL

In [1]:
from re_rl.dataset_generator import DatasetGenerator
from re_rl.tasks import ArithmeticTask, QuadraticTask, KinematicsTask, ALL_TASK_GENERATORS
from re_rl.tasks.physics.mechanics import DynamicsTask

## 1. Быстрый старт с reasoning_mode

In [3]:
generator = DatasetGenerator()

# Генерация с reasoning_mode=True (для обучения reasoning)
dataset = generator.generate_sft_dataset(
    task_types=["quadratic", "kinematics", "dynamics"],
    num_samples=30,
    language="ru",
    difficulties=[3, 5, 7],
    reasoning_mode=True,  # ← Включаем режим рассуждений!
)

print(f"Сгенерировано: {len(dataset)} примеров")
print("\n" + "="*60)
print("ПРИМЕР С reasoning_mode=True")
print("="*60)
print(f"input: {dataset[0]['input']}")
print(f"\noutput:\n{dataset[0]['output']}")

Генерация задач: 100%|██████████| 30/30 [00:00<00:00, 322.83задач/s]

Сгенерировано: 30 примеров

ПРИМЕР С reasoning_mode=True
input: Тело движется с начальной скоростью 14 м/с и ускорением 8 м/с². Какова скорость через 27 с?

output:
<think>
Дано:
  v₀ = 14 м/с
  a = 8 м/с²
  t = 27 с
Найти: v — конечная скорость
Формула: v = v₀ + at
Подстановка: v = 14 + 8 × 27
Вычисление: 14 + 216 = 230 м/с
Проверка размерности: [м/с] + [м/с²]×[с] = [м/с] + [м/с] = [м/с] ✓
</think>
<answer>230 м/с</answer>


In [ ]:
# Сравнение: БЕЗ reasoning_mode (plain text)
dataset_plain = generator.generate_sft_dataset(
    task_types=["dynamics"],
    num_samples=1,
    language="ru",
    reasoning_mode=True,  # ← БЕЗ тегов
)

print("="*60)
print("БЕЗ reasoning_mode (plain text)")
print("="*60)
print(dataset_plain[0]['output'])

Генерация задач: 100%|██████████| 1/1 [00:00<00:00, 9554.22задач/s]

БЕЗ reasoning_mode (plain text)
Формула: F = ma → a = F/m
Подстановка: a = 713 / 178
Вычисление: 713/178 = 4.0056 м/с²

Ответ: 4.0056 m/s^2


## 2. Генерация датасета на арифметику (20K примеров)

Генерируем большой датасет только на арифметику с разными уровнями сложности для тюнинга модели.

In [2]:
import os
import json
import random
from datetime import datetime

OUTPUT_DIR = "datasets"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Генерация 20K примеров арифметики
print("="*60)
print("ГЕНЕРАЦИЯ АРИФМЕТИЧЕСКОГО ДАТАСЕТА (20K примеров)")
print("="*60)

arith_gen = DatasetGenerator(output_dir=OUTPUT_DIR)

arithmetic_dataset = arith_gen.generate_sft_dataset(
    task_types=["arithmetic"],  # Только арифметика
    num_samples=30000,           # 20 тысяч примеров
    language="ru",
    difficulties=list(range(1, 11)),  # Все уровни 1-10
    reasoning_mode=True,         # С подробными рассуждениями
)

print(f"\nСгенерировано: {len(arithmetic_dataset)} примеров")

ГЕНЕРАЦИЯ АРИФМЕТИЧЕСКОГО ДАТАСЕТА (20K примеров)


Генерация задач: 100%|██████████| 30000/30000 [00:00<00:00, 36785.08задач/s]


Сгенерировано: 30000 примеров


In [ ]:
# Примеры разной сложности
print("="*60)
print("ПРИМЕРЫ РАЗНОЙ СЛОЖНОСТИ")
print("="*60)

for diff in [1, 3, 5, 7, 10]:
    examples = [ex for ex in arithmetic_dataset if ex['metadata']['difficulty'] == diff]
    if examples:
        print(f"\n--- Сложность {diff} ---")
        print(f"input: {examples[0]['input']}")
        out = examples[0]['output']
        print(f"output:\n{out[:600]}")
        if len(out) > 600:
            print("...")

In [3]:
# Сохраняем арифметический датасет
random.shuffle(arithmetic_dataset)
split_idx = int(len(arithmetic_dataset) * 0.9)
arith_train = arithmetic_dataset[:split_idx]
arith_eval = arithmetic_dataset[split_idx:]

timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
train_file = os.path.join(OUTPUT_DIR, f"arithmetic_train_{timestamp}.jsonl")
eval_file = os.path.join(OUTPUT_DIR, f"arithmetic_eval_{timestamp}.jsonl")

def save_jsonl(data, filepath):
    with open(filepath, 'w', encoding='utf-8') as f:
        for item in data:
            f.write(json.dumps(item, ensure_ascii=False) + '\n')

save_jsonl(arith_train, train_file)
save_jsonl(arith_eval, eval_file)

print("="*60)
print("АРИФМЕТИЧЕСКИЙ ДАТАСЕТ СОХРАНЁН!")
print("="*60)
print(f"\nФайлы:")
print(f"  Train: {train_file} ({len(arith_train)} примеров)")
print(f"  Eval: {eval_file} ({len(arith_eval)} примеров)")

from collections import Counter
diff_counts = Counter(item['metadata']['difficulty'] for item in arithmetic_dataset)
print(f"\nРаспределение по сложности:")
for diff in sorted(diff_counts.keys()):
    print(f"  Сложность {diff}: {diff_counts[diff]} примеров")

АРИФМЕТИЧЕСКИЙ ДАТАСЕТ СОХРАНЁН!

Файлы:
  Train: datasets/arithmetic_train_20260130_160050.jsonl (27000 примеров)
  Eval: datasets/arithmetic_eval_20260130_160050.jsonl (3000 примеров)

Распределение по сложности:
  Сложность 1: 3044 примеров
  Сложность 2: 2929 примеров
  Сложность 3: 3077 примеров
  Сложность 4: 3104 примеров
  Сложность 5: 2958 примеров
  Сложность 6: 3047 примеров
  Сложность 7: 2989 примеров
  Сложность 8: 2901 примеров
  Сложность 9: 2961 примеров
  Сложность 10: 2990 примеров


## 3. Все 81 тип задач

In [ ]:
from re_rl.tasks.physics.generators import ALL_PHYSICS_TASK_GENERATORS

all_tasks = sorted(ALL_TASK_GENERATORS.keys())
math_tasks = [t for t in all_tasks if t not in ALL_PHYSICS_TASK_GENERATORS]
physics_tasks = list(ALL_PHYSICS_TASK_GENERATORS.keys())

print(f"МАТЕМАТИКА ({len(math_tasks)}): {', '.join(sorted(math_tasks)[:10])}...")
print(f"\nФИЗИКА ({len(physics_tasks)}): {', '.join(sorted(physics_tasks)[:10])}...")
print(f"\n=== ИТОГО: {len(ALL_TASK_GENERATORS)} типов задач ===")

## 4. Примеры с reasoning_mode

In [ ]:
# Физика с reasoning_mode
print("="*60)
print("DynamicsTask с reasoning_mode=True")
print("="*60)

task = DynamicsTask(task_type="newton_second", m=5, F=25, reasoning_mode=True)
task.solve()
print(task.format_final_output())

In [ ]:
# Математика с reasoning_mode
print("="*60)
print("QuadraticTask с reasoning_mode=True")
print("="*60)

task = QuadraticTask(a=1, b=-5, c=6, reasoning_mode=True)
task.solve()
print(task.format_final_output())

## 5. Генерация полного датасета (все 81 тип)

In [ ]:
# Генерация полного датасета со всеми типами задач
full_gen = DatasetGenerator(output_dir=OUTPUT_DIR)

full_dataset = full_gen.generate_sft_dataset(
    task_types=None,  # Все 81 тип
    num_samples=5000,
    language="ru",
    difficulties=[3, 5, 7],
    reasoning_mode=True,
)

print(f"Сгенерировано: {len(full_dataset)} примеров")

# Сохранение
random.shuffle(full_dataset)
split_idx = int(len(full_dataset) * 0.9)

timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
save_jsonl(full_dataset[:split_idx], f"{OUTPUT_DIR}/full_train_{timestamp}.jsonl")
save_jsonl(full_dataset[split_idx:], f"{OUTPUT_DIR}/full_eval_{timestamp}.jsonl")

print(f"\nСохранено:")
print(f"  Train: {split_idx} примеров")
print(f"  Eval: {len(full_dataset) - split_idx} примеров")